## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

### Вывод

In [4]:
#P.S у меня нет русской клавиатуру,так как живу за границкй сейчас,все вбивалось в слепую, заранее принощу свои извинения за опечатки и ошибки если такие имеются 
'''import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer 
from pymystem3 import Mystem
from collections import Counter

russian_stemmer = SnowballStemmer('russian')
df = pd.read_csv('/datasets/data.csv')
#print(df.info())
# было бы не плохо посмотреть все уникальные значения, чтобы понять что то за таблица
print(df['children'].unique())
print(df['days_employed'].unique())
print(df['dob_years'].unique())
print(df['education'].unique())
print(df['education_id'].unique())
print(df['family_status'].unique())
print(df['family_status_id'].unique())
print(df['income_type'].unique())
print(df['debt'].unique())
print(df['total_income'].unique())
print(df['purpose'].unique())
#print(df.head(60)) #хочу посмотреть на таблицу 
#смортрим на дубликаты и пропущенные значения
#print("Дубликаты", df.duplicated().sum())
print("Пропуски", df.isnull().sum())
print("maximum", df['days_employed'].max())
print(df)'''

import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df)
df.to_csv('/datasets/data.csv')

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

OSError: [Errno 30] Read-only file system: '/datasets/data.csv'

итак имеем файл с 21525 строчками и 12 столбцами 
решил для себя еще проверить что тут за данные

И так я обнаружил NaN в столбце days_employed и total_income, проблема в том что days_employed принемает отрицательное значение, но разве рабочие дне могут быть отрицательными(меня это напрягает) ?

В education некотроые параметры написаны КАПСОМ 
В days_employed  значения NaN у пенсионеров и гос.служащих, пенсионреы не работают , а с гос.служ не ясно, возможно что в некоторых сферах они выходят раньше на пенсию

Пропущенные значения в days_employed  и total_income

Стаж по дням слишком большой, думаю пропустить пока этот столбец, вообще годов по идее достаточно 

Пустые значения total_income хочу поменять на медианную ЗП который у нас в списке

54 дубликата, терпимо, но желательно от них избваиться

Children 20 и -1 хочу поменять на 2 и 1 соотвественно, почему 20 на 2 ? Потомучто на полноразмерных клавиатурах(где распологаються цифры 0-9) 0 находиться прямо над 2, думаю оператор банка случайно нажал(надеюсь вы паняли где именно)

54 Дубликата, в теории терпимо, но у нас тут банк, и по это каждая строчка важна



### Шаг 2. Предобработка данных

### Обработка пропусков

In [17]:
#Меняем значения, исправляем NaN в таблице по ЗП
#total_income_median = df['total_income'].median()
#days_employed_median = df['days_employed'].median()
#df['total_income'] = df['total_income'].fillna(value = total_income_median)
#почему я так сделал? Ради того чтобы не потерять 2к данных при анализе
#df['days_employed'] = df['days_employed'].fillna(value = days_employed_median)
#хочу поменять -1 и 20 на 1 и 2 соотвественно
#df['children'] = df['children'].replace( -1 , 1)
#df['children'] = df['children'].replace( 20 , 2)
#пропушу пока тоже значения NaN в столбике про стаж 
# это я просто проверяю для себя print(df.head(60))#['total_income']

#print(df.info())
#print(df)
median_values = pd.pivot_table(
    df,
    index=['education', 'income_type'],
    values=['days_employed', 'total_income'],
    aggfunc=np.median
)

median_values #из других уроков я узнал что можно делать так, решил немного пересмотреть прошлый способ избавления от пропусков

def fill_days_employed(row):
    return median_values.loc[row['education']].loc[row['income_type']]['days_employed']
# думаю написать функцию заполнения пропусков дней стажа на основе сводной таблицы
df.loc[
    df['days_employed'].isnull(), 'days_employed'
] = df.apply(fill_days_employed, axis=1)
# восстановим пропуски при помощи вычисленных значений
def fill_total_income(row):
    return median_values.loc[row['education']].loc[row['income_type']]['total_income']
#функция для заполнения пропусков уровня дохода на основе сводной таблицы
df.loc[
    df['total_income'].isnull(), 'total_income'
] = df.apply(fill_total_income, axis=1)
# делаю пропуски при помощи вычисленний

df = df[(df['children'] >= 0) & (df['children'] <= 5)]
df.sort_values('children')['children'].value_counts().sort_index()
#избавляемся от ошибок
df.shape[0]

21402

### Вывод

Я заменил все типы NaN в двух столбцах, в следушем пункте буду менять данные по детям в стобце про детей
Нужно что-то делать со стажем, пока идей нет, не могу понять логики их порчи
Думаю вообще стаж по дням не брать, окей если нет точного стажа то что мы будем использывать в будещи ?
Ответ на это вопрос в голову приходит только сам возраст кто хочет взять кредит, данные по стажу слишком испорчены.
Я думаю возроста вполне хватит для учета,ведь не может человек в 30+ лет не работать и не иметь доход, окей а молодая катергория граждан 21+? Ну они указали в графе доход, от дохода и будем в основном исходить
Поменял 20 и -1 соотвественно
Исправил по замечанию, вроде не потерял данные, заменил на медиану возраст, что бы они не пропали
Я избавилися от пропусков, оставив только необходимые для анализа данные

### Замена типа данных

In [23]:
df['days_employed'] = df['days_employed'].astype('int').apply(abs)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

изменил количество дней стажа в целочисленный тип при помощи метода astype, столбец избавился от отрицательных значений, наверное так будет лучше

In [20]:
###Обработка дубликатов

In [24]:
df['education'].str.lower()#избавился от капса
#на первый взгляд вроде они и не похожы на дубликаты
#я думаю что данные это не дубликаты, это очень похожие пользователи чьи дааные совпали
#df.drop_duplicates().reset_index(drop=True)
#df.duplicates().sum()
# не понимаю почему НаН еще сохранился
df.sort_values('education')['education'].value_counts()
df.duplicated().sum()

54

### Вывод

Вроде все дубликаты устранены

### Лемматизация

In [31]:
# хочу использовать библиотекой Mystem 
m = Mystem()
purposes = []
#хочу найти самые частовстречаемые слова в леммах и на основе результата определить категории целей кредита
for p in df['purpose']:
    lemma = m.lemmatize(p)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)

#print(Counter(purposes))

# во множестве я заметил общие слова типа "покупка", "операция", "строительство" и тд
# если их откинуть, то наиболее частовстречаемыми категориями будут
purpose_category = [
    'свадьба',
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование'
]
#хочу написать ф-ию
def categorize_purpose(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in purpose_category:
            if category in lemma:
                return category
        
#новый столбец с категориями целей кредита
df['purpose_category'] = df['purpose'].apply(categorize_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба


### Вывод

я пять категорий целей кредита – читать и анализировать такие данные будет легче

### Категоризация данных


In [35]:

'''Распределим по наличию детей

0 – бездетная
1 – однодетная
2 – малодетная
3+ – многодетная'''
def categorize_children(count):
    categories = {
        0: 'бездетная',
        1: 'однодетная',
        2: 'малодетная'
    }

    return categories.get(count, 'многодетная')

#df['children_category'] = df['children'].apply(categorize_children)

'''Распределим по семейному статосу

в браке – женат / замужем, гражданский брак
не в браке – не женат / не замужем, в разводе, вдовец / вдова'''
def categorize_family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'
#df['family_status_category'] = df['family_status'].apply(categorize_family_status)

#уровень ЗП

timedian = df['total_income'].median()
def categorize_total_income(value):
    if value < 50_000:
        return 'низкий'
    elif value <= timedian:
        return 'средний'
    elif value < 350_000:
        return 'выше среднего'
    else:
        return 'высокий'
   
    
df['children_category'] = df['children'].apply(categorize_children)   
df['family_status_category'] = df['family_status'].apply(categorize_family_status)    
df['total_income_category'] = df['total_income'].apply(categorize_total_income)    

### Вывод я сделал категории по трем параметрам по детям\семье и доходу

### Шаг 3. Ответьте на вопросы

<font color='blue'>В ответах на вопросы нужно посчитать вероятность просрочки для каждой категории (в зависимости от вопроса), т.е. отношение людей с просрочкой из категории ко всем людям из категории.</font>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [36]:
pd.pivot_table(df, index='children_category', values='debt')

,debt
children_category,
бездетная,0.075129
малодетная,0.094404
многодетная,0.079523
однодетная,0.092154


### Вывод

Прямой зависимости между наличием детей и возвратом кредита в срок нет до 9.4% клиентов имели задолжности по кредитам, независимо от того были у них дети или нет
Бездетные семьи чаще выплачивают кредит (ожидаемо): ~7.5%
Что интересно, многодетные семьи реже остаются в должниках ~8.2%, чем семьи с одним ребенком ~9.2% или двумя ~9.4%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
pd.pivot_table(df, index='family_status_category', values='debt')

,debt
family_status_category,
в браке,0.075202
не в браке,0.088573


### Вывод

Клиенты, состоящие в браке, на 1.3% реже остаются в должниках

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
pd.pivot_table(df, index='total_income_category', values='debt')

,debt
total_income_category,
высокий,0.074208
выше среднего,0.080753
низкий,0.061828
средний,0.083942


### Вывод

Возврат кредита зависит от уровня дохода клиента чем выше доход, тем меньше % задолжностей
Но я заметил что люди с низким уровнем дохода имеют самый низкий процент задолжностей по кредитам ~6.2%, а вот люди со средним  самый высокий ~8.5%

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
pd.pivot_table(df, index='purpose_category', values='debt')

### Вывод

Недвижемость и свадьба более приориттеная чем автомобль и образвоания

### Шаг 4. Общий вывод

Я думаю отделу выдочи кредита безопаснее всего выдавать кредиты на покупку жилья, недвижимости или свадьбу по ним меньшее количество задолжностей

Для более детального ответа необходимо рассмотреть следующие данные о клиенте

Наличие детей
наличие семьи
уровнь дохода
цель кредита лучше жилье или недвижимость

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.